In [ ]:
%run ../../lib/basic-import.ipynb
%run ../../lib/plot-functions.ipynb

In [ ]:
#print(part_df.keys())
#epic_df = pd.concat(part_df, axis=1, keys=[a[:-4] for a in participant_files])
epic_df = pd.concat(all_messages, axis=1, keys=[a[:-4] for a in participant_files])
epic_2df = pd.concat(all_hrv, axis=1, keys=[a[:-4] for a in participant_files])

In [ ]:
epic_2df['50338'].duplicated(subset=['timestamp']).sum()
epic_df['50338'].duplicated(subset=['timestamp']).sum()
epic_df['50338'].duplicated(subset=['timestamp','type']).sum()
only_dup_epic_338 = epic_df['50338'][epic_df['50338','timestamp'].duplicated()]
print(len(only_dup_epic_338))
only_dup_epic_338.duplicated(subset=['type']).sum()
compression_opts = dict(method='zip', archive_name='out.csv')
only_dup_epic_338.to_csv('out.zip', compression=compression_opts)

In [ ]:
def pd_plot_lap_hrv():
    add_df = pd.concat(all_dfs, axis=1, keys=[a[:-4] for a in participant_files])

    #for now, we are only looking at 50338
    add_df_338 = add_df['50338']
    # issue #15
    add_df_338 = add_df_338[add_df_338['timestamp'].notnull()] 

    tempAdd_df = [] # should be renamed lapRange_df_li
    lapDates = []   # should be renamed lapTimestamp_li
    lapCount = 0

    # Find the timestamps where lap increases
    for i in add_df_338.index:
        if add_df_338.loc[i,'lap'] > lapCount:
            lapDates.append(add_df_338.loc[i,'timestamp'])
            lapCount += 1

    # Find +-30 s from lap increase, and store all rows in range
    add_df_338.set_index('timestamp', inplace=True)
    timedelta_30s = pd.Timedelta("00:00:30")
    for i in range(len(lapDates)):
        startRow = add_df_338.index.get_loc(lapDates[i]-timedelta_30s)
        endRow = add_df_338.index.get_loc(lapDates[i]+timedelta_30s)
        tempAdd_df.append(add_df_338.iloc[startRow:endRow])
    
    # Create plot for every stop
    for i in range(len(lapDates)):
        fig, ax = plt.subplots()

        trans = ax.get_xaxis_transform()

        ax2 = ax.twinx()
        ax3 = ax.twinx()
        ax4 = ax.twinx()
        ax5 = ax.twinx()
        ax6 = ax.twinx()

        rightPosition = 1.00
        leftPosition = -0.10

        rightPosition = ax_modification(ax2,'right', rightPosition)
        rightPosition = ax_modification(ax3,'right', rightPosition)
        leftPosition = ax_modification(ax4,'left', leftPosition)
        rightPosition = ax_modification(ax5,'right', rightPosition)
        leftPosition = ax_modification(ax6,'left', leftPosition)
        
        tempAdd_df[i]['accumulated_power'].plot(ax=ax, color='red',legend=False, title="all laps +-30, pd 50338")
        tempAdd_df[i]['speed'].plot(ax=ax2, color='blue',legend=False)
        tempAdd_df[i]['power'].plot(ax=ax3, color='green',legend=False)
        tempAdd_df[i]['altitude'].plot(ax=ax4, color='orange',legend=False)
        tempAdd_df[i]['distance'].plot(ax=ax5, color='lightcoral',legend=False)
        tempAdd_df[i]['heart_rate'].plot(ax=ax6, color='purple',legend=False)    
        ax.axvline((lapDates[i]), color='black', linestyle='--', lw=2)
        ax.text((lapDates[i]),.5,(lapDates[i]),transform=trans)
        
        ax_style(ax,'Accumulated Power','red')
        ax_style(ax2,'Speed','blue')
        ax_style(ax3,'Power','green')
        ax_style(ax4,'Altitude','orange')
        ax_style(ax5,'Distance','lightcoral')
        ax_style(ax6,'Heartrate','purple')

In [ ]:
pd_plot_lap_hrv